# Avalanche Historical Staking Distribution

### Setup

In [1]:
import datetime
import requests
import pandas as pd

In [2]:
# First date with any data
start_date = datetime.date(2020,9,23)

# Run until today
end_date = datetime.date.today()

### Getting Data

In [5]:
# inclusive for loop from start_date to end_date
rank = []
cumulative = []
for date in pd.date_range(start_date,end_date):
    
    # Doing this so we have something as a progress bar
    date_str = str(date.date())
    print(date_str)
    
    # pulled from: https://ava-labs-inc.metabaseapp.com/public/dashboard/249aea4f-6f83-45f4-8ed7-5cbdf9e9b026?single_date=2022-01-25
    # {data: {rows: [[ID, stake amount, rank], ...]}}
    rank_weight_url = f'https://ava-labs-inc.metabaseapp.com/api/public/dashboard/249aea4f-6f83-45f4-8ed7-5cbdf9e9b026/card/234?parameters=%5B%7B%22type%22%3A%22date%2Fsingle%22%2C%22value%22%3A%22{date_str}%22%2C%22target%22%3A%5B%22variable%22%2C%5B%22template-tag%22%2C%22FILTER_DATE%22%5D%5D%7D%5D'
    # {data: {rows: [[rank, weight], ...]}}
    cumulative_weight_url = f'https://ava-labs-inc.metabaseapp.com/api/public/dashboard/249aea4f-6f83-45f4-8ed7-5cbdf9e9b026/card/233?parameters=%5B%7B%22type%22%3A%22date%2Fsingle%22%2C%22value%22%3A%22{date_str}%22%2C%22target%22%3A%5B%22variable%22%2C%5B%22template-tag%22%2C%22FILTER_DATE%22%5D%5D%7D%5D'

    # get lists
    rank_weight_list = requests.get(rank_weight_url).json()['data']['rows']
    cumulative_weight_list = requests.get(cumulative_weight_url).json()['data']['rows']
    
    # stick on bigger lists
    rank.append({'date': date_str, 'rank': rank_weight_list})
    cumulative.append({'date': date_str, 'cumulative': cumulative_weight_list})

2020-09-23
2020-09-24
2020-09-25
2020-09-26
2020-09-27
2020-09-28
2020-09-29
2020-09-30
2020-10-01
2020-10-02
2020-10-03
2020-10-04
2020-10-05
2020-10-06
2020-10-07
2020-10-08
2020-10-09
2020-10-10
2020-10-11
2020-10-12
2020-10-13
2020-10-14
2020-10-15
2020-10-16
2020-10-17
2020-10-18
2020-10-19
2020-10-20
2020-10-21
2020-10-22
2020-10-23
2020-10-24
2020-10-25
2020-10-26
2020-10-27
2020-10-28
2020-10-29
2020-10-30
2020-10-31
2020-11-01
2020-11-02
2020-11-03
2020-11-04
2020-11-05
2020-11-06
2020-11-07
2020-11-08
2020-11-09
2020-11-10
2020-11-11
2020-11-12
2020-11-13
2020-11-14
2020-11-15
2020-11-16
2020-11-17
2020-11-18
2020-11-19
2020-11-20
2020-11-21
2020-11-22
2020-11-23
2020-11-24
2020-11-25
2020-11-26
2020-11-27
2020-11-28
2020-11-29
2020-11-30
2020-12-01
2020-12-02
2020-12-03
2020-12-04
2020-12-05
2020-12-06
2020-12-07
2020-12-08
2020-12-09
2020-12-10
2020-12-11
2020-12-12
2020-12-13
2020-12-14
2020-12-15
2020-12-16
2020-12-17
2020-12-18
2020-12-19
2020-12-20
2020-12-21
2020-12-22

### Clean up data

In [7]:
rank_df = pd.DataFrame(rank)
cumulative_df = pd.DataFrame(cumulative)

In [9]:
cumulative_df

,date,cumulative
0,2020-09-23,"[[1, 0.09179796638094224], [2, 0.1789408852731..."
1,2020-09-24,"[[1, 0.0705515181776062], [2, 0.14110303635521..."
2,2020-09-25,"[[1, 0.06418845583470877], [2, 0.1283769116694..."
3,2020-09-26,"[[1, 0.05044181707656916], [2, 0.1008836341531..."
4,2020-09-27,"[[1, 0.04813200586552204], [2, 0.0962640117310..."
...,...,...
485,2022-01-21,"[[1, 0.012623944962451486], [2, 0.025247889924..."
486,2022-01-22,"[[1, 0.012634852948199132], [2, 0.025269705896..."
487,2022-01-23,"[[1, 0.012623280435964317], [2, 0.025246560871..."
488,2022-01-24,"[[1, 0.0126207879973492], [2, 0.02524157599469..."


### Nakamoto Coefficient

In [12]:
dict_list = []
for index, row in cumulative_df.iterrows():
    date = row['date']
    cumulative_list = row['cumulative']
    total_validators = len(cumulative_list)
    
    count = 0
    total_share = cumulative_list[count][1]
    while total_share < 0.333:
        count+=1
        total_share = cumulative_list[count][1]
    
    one_third = cumulative_list[count][0]
    
    
    nakamoto_coeff = one_third / total_validators
    
    entry = {'date': date, 'nakmoto': nakamoto_coeff, 'one_third': one_third, 'validators': total_validators}
    dict_list.append(entry)

In [17]:
nakamoto_df = pd.DataFrame(dict_list)
nakamoto_df.set_index('date', inplace=True)
nakamoto_df.to_csv('avalanche_nakamoto.csv')
nakamoto_df

,nakmoto,one_third,validators
date,,,
2020-09-23,0.022831,5,219
2020-09-24,0.017483,5,286
2020-09-25,0.018576,6,323
2020-09-26,0.019499,7,359
2020-09-27,0.018182,7,385
...,...,...,...
2022-01-21,0.022005,27,1227
2022-01-22,0.021792,27,1239
2022-01-23,0.021916,27,1232
